In [1]:
# # Step 1: Install Required Libraries
# !pip install tensorflow numpy


In [2]:
#Step 2: Import Libraries
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import text_to_word_sequence
import numpy as np


In [3]:
#Step 3: Load and Prepare the Dataset
# Load IMDB dataset
num_words = 10000  # Only use the top 10,000 most frequent words
maxlen = 200       # Pad sequences to max length

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=num_words)

# Pad sequences so they are all the same length
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)


In [4]:
# #Step 4: Build the RNN Model
model = Sequential([
    Embedding(input_dim=num_words, output_dim=32),  # input_length removed
    SimpleRNN(units=32, activation='tanh'),
    Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [5]:
#Step 5: Train the Model
model.fit(x_train, y_train, epochs=3, batch_size=64, validation_split=0.2)


Epoch 1/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 21s 54ms/step - accuracy: 0.5498 - loss: 0.6811 - val_accuracy: 0.6740 - val_loss: 0.5969
Epoch 2/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 15s 47ms/step - accuracy: 0.7811 - loss: 0.4657 - val_accuracy: 0.7506 - val_loss: 0.5210
Epoch 3/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 14s 44ms/step - accuracy: 0.8698 - loss: 0.3236 - val_accuracy: 0.8264 - val_loss: 0.4264


In [6]:
#Step 6: Evaluate the Model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_acc:.2f}")



782/782 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accuracy: 0.8225 - loss: 0.4387
Test Accuracy: 0.82


In [7]:
#Step 7: Make Predictions
# Reverse lookup to convert indices to words
word_index = imdb.get_word_index()
index_word = {v + 3: k for k, v in word_index.items()}
index_word[0] = '<PAD>'
index_word[1] = '<START>'
index_word[2] = '<UNK>'
index_word[3] = '<UNUSED>'

# Decode the review
def decode_review(encoded_review):
    return ' '.join([index_word.get(i, '?') for i in encoded_review])

sample = x_test[0]
print("Review:")
print(decode_review(sample))

prediction = model.predict(np.expand_dims(sample, axis=0))[0][0]
print(f"\nPredicted Sentiment: {'Positive' if prediction > 0.5 else 'Negative'} (score={prediction:.2f})")



Review:
<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <START> please give this one a miss br br <UNK> <UNK> and the rest of the cast rendered terrible performances the show is flat flat flat br br i don't know how michael madison could have allowed this 

In [8]:
#Predict Custom Input
def encode_review(text):
    words = text_to_word_sequence(text)
    encoded = [1]  # 1 is the <START> token
    for word in words:
        index = word_index.get(word)
        if index is not None and index < num_words:
            encoded.append(index + 3)
        else:
            encoded.append(2)  # <UNK>
    return pad_sequences([encoded], maxlen=maxlen)

In [9]:
# Custom review
your_review = "This movie was absolutely wonderful and inspiring"
encoded = encode_review(your_review)

In [10]:
## Prediction
pred = model.predict(encoded)[0][0]
print(f"\nYour review: {your_review}")
print(f"Predicted Sentiment: {'Positive' if pred > 0.5 else 'Negative'} (score={pred:.2f})")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step

Your review: This movie was absolutely wonderful and inspiring
Predicted Sentiment: Positive (score=0.85)


In [11]:
#
